In [ ]:
# импорт необходимых библиотек

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split,  RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, roc_auc_score

In [ ]:
# загружаем данные

train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

Выведем информацию о наборе train.csv

In [ ]:
# первые 5 записей набора данных

train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# размер данных

train_data.shape

(891, 12)

In [ ]:
# общая информация о фрейме данных

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Таким образом, набор train.csv содержит 891 запись, каждая из которых описывается 12 признаками:

1.   PassengerId - уникальный номер пользователя;
2.   Survived - указатель выжил или нет пассажир;
3.   Pclass - класс билета;
4.   Name - имя пассажира;
5.   Sex - пол;
6.   Age - возраст;
7.   SibSp - количество братьев и сестер / супругов на борту «Титаника»;
8.   Parch - количество родителей / детей на борту «Титаника»;
9.   Ticket - номер билета;
10.   Fare - пассажирский тариф;
11.   Cabin - номер каюты;
12.   Embarked - порт погрузки. 

Типы данных признаков: вещественный - float64 (2 признака), целочисленный - int64 (5 признаков), строковый - object (5 признаков).


In [ ]:
# удаление дубликатов

train_data = train_data.drop_duplicates()
print(train_data.shape)

(891, 12)


Исследуемый набор данных не содержит дубликатов.

In [ ]:
# основные статистические характеристики каждого числового признака (int64 и float64)

train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


В полученной таблице содержится основная информация о числовых признаках датасета: количество записей, среднее, медианное, максимальное и минимальное значения, стандартное отклонение, 0.25 и 0.75 квартили.

То есть возвращается статистическая информация, которая дает представление о распределении значений. 

In [ ]:
# статистика для нечисловых признаков

train_data.describe(include=['object'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


В полученной таблице содержится информация о строковых признаках датасета: количество записей, количество уникальных значений, наиболее распространенное значение и его частота.

In [ ]:
# концентрация пропусков в каждом столбце (в процентах)

train_data.isnull().mean() * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

Три показателя содержат пропуски в данных:

1.   Age (возраст) - порядка 19,9%;
2.   Cabin (номер каюты) - порядка 77,1%;
3.   Embarked (порт погрузки) - 0,22%.

Больше всего пропуксов в переменной 'Cabin'.
Отметим, что концентрация пропусков для столбца 'Cabin' свыше 70%, поэтому требуется его удаление, удаление остальных столбцов не требуется.

Кроме того, удалим столбцы 'PassengerId', 'Name', 'Ticket', которые могут привести к переобучению.

In [ ]:
# удаление столбцов

data = train_data.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'])

Пропущенные значения в столбце 'Age' заменим средним значением.

Пропущенные значения в столбце 'Embarked' наиболее часто встречающимся значением.

In [ ]:
data['Age'].fillna(data['Age'].mean(), inplace = True)
data['Embarked'].replace(np.nan, 'S', inplace = True)

In [ ]:
# Числовые признаки
num_cols = [
    'Pclass', 
    'Age', 
    'SibSp', 
    'Parch', 
    'Fare'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'Embarked'
]

In [ ]:
X = data.iloc[:, 1:]
y = data['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# преобразование признаков

prep_pipeline = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop='if_binary', sparse=False), cat_cols)
    ])

In [ ]:
# создание модели LogisticRegression
clf_lr = make_pipeline(
                    prep_pipeline,
                    LogisticRegression()
                  )

# обучение модели
clf_lr.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [ ]:
print("LogisticRegression\n")
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
  y_pred_lr = clf_lr.predict(X)
  print(f'MSE {label}={mean_squared_error(y, y_pred_lr):.2f}')
  print(f'MAE {label}={mean_absolute_error(y, y_pred_lr):.2f}')
  print(f'R2 {label}={r2_score(y, y_pred_lr):.2f}')
  print(f'Accuracy {label}={accuracy_score(y, y_pred_lr):.2f}')
  print(f'Precision {label}={precision_score(y, y_pred_lr):.2f}')
  print(f'Recall {label}={recall_score(y, y_pred_lr):.2f}')
  print(f'ROC-AUC {label}={roc_auc_score(y, y_pred_lr):.2f}')
  print()   

LogisticRegression

MSE train=0.20
MAE train=0.20
R2 train=0.16
Accuracy train=0.80
Precision train=0.76
Recall train=0.69
ROC-AUC train=0.78

MSE test=0.19
MAE test=0.19
R2 test=0.22
Accuracy test=0.81
Precision test=0.79
Recall test=0.74
ROC-AUC test=0.80



In [ ]:
# создание модели KNeighborsClassifier
clf_knn = make_pipeline(
                    prep_pipeline,
                    KNeighborsClassifier()
                  )

# обучение модели
clf_knn.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [ ]:
print("KNeighborsClassifier\n")
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
  y_pred_knn = clf_knn.predict(X)
  print(f'MSE {label}={mean_squared_error(y, y_pred_knn):.2f}')
  print(f'MAE {label}={mean_absolute_error(y, y_pred_knn):.2f}')
  print(f'R2 {label}={r2_score(y, y_pred_knn):.2f}')
  print(f'Accuracy {label}={accuracy_score(y, y_pred_knn):.2f}')
  print(f'Precision {label}={precision_score(y, y_pred_knn):.2f}')
  print(f'Recall {label}={recall_score(y, y_pred_knn):.2f}')
  print(f'ROC-AUC {label}={roc_auc_score(y, y_pred_knn):.2f}')
  print()

KNeighborsClassifier

MSE train=0.14
MAE train=0.14
R2 train=0.39
Accuracy train=0.86
Precision train=0.85
Recall train=0.75
ROC-AUC train=0.84

MSE test=0.18
MAE test=0.18
R2 test=0.24
Accuracy test=0.82
Precision test=0.81
Recall test=0.73
ROC-AUC test=0.80



Выводы:

- по совокупности исследуемых метрик метод KNN показал результаты лучше, чем метод LR;
- на тестовой выборке метод LR показал результаты лучше, чем на обучающей выборке;
- на тестовой выборке метод KNN показал результаты хуже, чем на обучающей выборке.

Подбор гиперпараметров для обеих моделей с помощью RandomizedSearchCV.

In [ ]:
clf_lr = make_pipeline(
                    prep_pipeline,
                    LogisticRegression()
                  )

params_lr = { 'logisticregression__C': np.linspace(0.01, 50, 5000), #[500, 50, 100, 10, 5, 1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001],
              'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

search_lr = RandomizedSearchCV(clf_lr, params_lr, n_iter=30, n_jobs=-1, cv=5)

search_lr.fit(X_train, y_train)

print("Best estimator's parameters:", search_lr.best_params_)

Best estimator's parameters: {'logisticregression__solver': 'saga', 'logisticregression__C': 0.14}


In [ ]:
# обучим модели на полученных параметрах

# создание модели LogisticRegression
clf_lr = make_pipeline(
                    prep_pipeline,
                    LogisticRegression(solver='saga', C=0.14)
                  )

# обучение модели
clf_lr.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression',
                 LogisticRegression(C=0.14, solver='saga'))])

In [ ]:
print("LogisticRegression\n")
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
  y_pred_lr = clf_lr.predict(X)
  print(f'MSE {label}={mean_squared_error(y, y_pred_lr):.2f}')
  print(f'MAE {label}={mean_absolute_error(y, y_pred_lr):.2f}')
  print(f'R2 {label}={r2_score(y, y_pred_lr):.2f}')
  print(f'Accuracy {label}={accuracy_score(y, y_pred_lr):.2f}')
  print(f'Precision {label}={precision_score(y, y_pred_lr):.2f}')
  print(f'Recall {label}={recall_score(y, y_pred_lr):.2f}')
  print(f'ROC-AUC {label}={roc_auc_score(y, y_pred_lr):.2f}')
  print()   

LogisticRegression

MSE train=0.19
MAE train=0.19
R2 train=0.17
Accuracy train=0.81
Precision train=0.78
Recall train=0.68
ROC-AUC train=0.78

MSE test=0.20
MAE test=0.20
R2 test=0.19
Accuracy test=0.80
Precision test=0.77
Recall test=0.74
ROC-AUC test=0.80



In [ ]:
clf_knn = make_pipeline(
                    prep_pipeline,
                    KNeighborsClassifier()
                  )

params_knn = {
    'kneighborsclassifier__n_neighbors': np.arange(1, 11), 
    'kneighborsclassifier__metric': ['manhattan', 'euclidean', 'cityblock', 'l1', 'l2', 'minkowski'],
    'kneighborsclassifier__weights': ['uniform', 'distance'],
    'kneighborsclassifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

search_knn = RandomizedSearchCV(clf_knn, params_knn, n_iter=30, n_jobs=-1, cv=5)

search_knn.fit(X_train, y_train)

print("Best estimator's parameters:", search_knn.best_params_)

Best estimator's parameters: {'kneighborsclassifier__weights': 'uniform', 'kneighborsclassifier__n_neighbors': 8, 'kneighborsclassifier__metric': 'cityblock', 'kneighborsclassifier__algorithm': 'ball_tree'}


In [ ]:
# создание модели KNeighborsClassifier
clf_knn = make_pipeline(
                    prep_pipeline,
                    KNeighborsClassifier(weights='uniform', n_neighbors=8, metric='cityblock', algorithm='ball_tree')
                  )

# обучение модели
clf_knn.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['Sex', 'Embarked'])])),
                ('kneighborsclassifier',
                 KNeighborsClassifier(algorithm='ball_tree', metric='cityblock',
                                      n_neighbors=8))])

In [ ]:
print("KNeighborsClassifier\n")
for X, y, label in zip([X_train, X_test], [y_train, y_test], ['train', 'test']):
  y_pred_knn = clf_knn.predict(X)
  print(f'MSE {label}={mean_squared_error(y, y_pred_knn):.2f}')
  print(f'MAE {label}={mean_absolute_error(y, y_pred_knn):.2f}')
  print(f'R2 {label}={r2_score(y, y_pred_knn):.2f}')
  print(f'Accuracy {label}={accuracy_score(y, y_pred_knn):.2f}')
  print(f'Precision {label}={precision_score(y, y_pred_knn):.2f}')
  print(f'Recall {label}={recall_score(y, y_pred_knn):.2f}')
  print(f'ROC-AUC {label}={roc_auc_score(y, y_pred_knn):.2f}')
  print()

KNeighborsClassifier

MSE train=0.17
MAE train=0.17
R2 train=0.29
Accuracy train=0.83
Precision train=0.86
Recall train=0.67
ROC-AUC train=0.80

MSE test=0.21
MAE test=0.21
R2 test=0.15
Accuracy test=0.79
Precision test=0.81
Recall test=0.65
ROC-AUC test=0.77



Используемый в работе метод подбора гиперпараметров моделей RandomizedSearchCV() позволяет осуществить поиск по параметрам, где каждый параметр выбирается из распределения по возможным значениям параметров.

То есть данный метод позволяет довольно грубо исследовать широкие диапазоны значений. 

Отметим, что при различных запусках метода можно получить различные наборы гиперпараметров методов.

Выводы:

- на обучающей выборке по совокупности исследуемых метрик метод KNN показал результаты лучше, чем метод LR;
- на тестовой выборке метод LR показал результаты примерно одинаковые (изменения ±0.01-0.02), как на обучающей выборке, при этом достаточно увеличилось значение матрики Recall - на 0.06;
- на тестовой выборке метод KNN показал результаты хуже, чем на обучающей выборке;
- подобранные с помощью метода RandomizedSearchCV() гиперпараметры позволили улучшить значения метрик на обучающей выборке для метода LR в сравнении с базовой моделью, однако на тестовой выборке показания метрик ухудшились, как для метода LR, так и для метода KNN.

Предсказания моделей (с подобранными гиперпараметрами) на наборе данных test.csv.

In [ ]:
test_data = test_data.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'])

In [ ]:
test_data['Age'].fillna(test_data['Age'].mean(), inplace = True)
test_data['Embarked'].replace(np.nan, 'S', inplace = True)

In [ ]:
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace = True)

In [ ]:
y_pred_knn = clf_knn.predict(test_data)
y_pred_lrr = clf_lr.predict(test_data)